In [1]:
import pandas as pd 
import numpy as np  
import ast
import pyarrow
from textblob import TextBlob

from sklearn.metrics.pairwise        import cosine_similarity
from sklearn.metrics.pairwise        import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer

## LECTURA DE LOS ARCHIVOS

In [56]:
df = pd.read_json('steam_games.json', orient='records', lines=True, convert_dates=True)
df.tail()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [52]:
lista = []
df2 = '/Users/danielh/Desktop/P HENRY/users_items.json'


#Abrir el archivo y procesar cada línea
with open(df2, 'r', encoding='utf-8') as df:
    for i in df:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(i)
            lista.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {i}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df2 = pd.DataFrame(lista)

In [53]:
df2.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [51]:
lista = []
df3 = 'user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(df3, 'r', encoding='utf-8') as df:
    for i in df:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(i)
            lista.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {i}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
df3 = pd.DataFrame(lista)

In [54]:
df3.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


# PREPARANDO EL PRIMER ARCHIVO

In [57]:
df.rename(columns = {'id':'item_id'}, inplace = True)


In [58]:
df1 = df.drop(columns=['app_name', 'url', 'reviews_url', 'specs', 'early_access'], axis=1)

In [60]:
df1['genres'].str.strip("[]").str.replace("'", "")

0                                        None
1                                        None
2                                        None
3                                        None
4                                        None
                         ...                 
120440    Casual, Indie, Simulation, Strategy
120441                Casual, Indie, Strategy
120442              Indie, Racing, Simulation
120443                          Casual, Indie
120444                                   None
Name: genres, Length: 120445, dtype: object

In [59]:
df1[["genres", "tags", "title", "developer"]] = df1[["genres", "tags", "title", "developer"]].astype(str)

In [17]:
df1 = df1.dropna()
df1

,publisher,genres,title,release_date,tags,price,item_id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",1.99,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530.0,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660.0,Laush Dmitriy Sergeevich


In [23]:
def convertir_a_entero_o_cero(df1):
    try:
        return float(df1)
    except (ValueError, TypeError):
        return 0

# Aplicar la función a la columna 'columna_strings' del DataFrame
df1['price'] = df1['price'].apply(convertir_a_entero_o_cero)

In [24]:
df1

,publisher,genres,title,release_date,tags,price,item_id,developer
88310,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980.0,Secret Level SRL
88312,Poolians.com,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290.0,Poolians.com
88313,彼岸领域,"['Action', 'Adventure', 'Casual']",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域
88315,Trickjump Games Ltd,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...",3.99,772540.0,Trickjump Games Ltd
...,...,...,...,...,...,...,...,...
120439,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,"[Action, Indie, Casual, Violent, Adventure]",1.99,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530.0,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660.0,Laush Dmitriy Sergeevich


In [18]:
df1["item_id"] = df1["item_id"].astype(float)


In [25]:
df1["price"] = df1["price"].astype(float)

In [27]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22845 entries, 88310 to 120443
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22845 non-null  object 
 1   genres        22845 non-null  object 
 2   title         22845 non-null  object 
 3   release_date  22845 non-null  object 
 4   tags          22845 non-null  object 
 5   price         22845 non-null  float64
 6   item_id       22845 non-null  float64
 7   developer     22845 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.6+ MB


In [28]:
df1['release_date'] = pd.to_datetime(df1['release_date'], errors='coerce')

In [29]:
df1.to_parquet('I_items.parquet', index=False)

# Preparacion del 2 archivo

In [30]:
df2.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [31]:
df2.shape

(88310, 5)

In [32]:
df2= df2.explode(['items'])
ddf2 = pd.json_normalize(df2['items']).set_index(df2['items'].index)
df02= pd.concat([df2, ddf2], axis=1)

In [33]:
df02.tail()

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330,All Is Dust,0.0,0.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",388490,One Way To Die: Steam Edition,3.0,3.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",521570,You Have 10 Seconds 2,4.0,4.0
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",519140,Minds Eyes,3.0,3.0
88309,edward_tremethick,0,76561198331598578,http://steamcommunity.com/id/edward_tremethick,NaN,NaN,NaN,NaN,NaN


In [34]:
df02["item_id"]= df02["item_id"].astype(float)

In [35]:
df02.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5170015 entries, 0 to 88309
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   items             object 
 5   item_id           float64
 6   item_name         object 
 7   playtime_forever  float64
 8   playtime_2weeks   float64
dtypes: float64(3), int64(1), object(5)
memory usage: 394.4+ MB


In [36]:
df02.to_parquet('u_items.parquet', index=False)

EXPORTACION DEL SEGUNDO ARCHIVO A CSV

In [37]:
df3= df3.explode(['reviews'])
ddf3 = pd.json_normalize(df3['reviews']).set_index(df3['reviews'].index)
df03= pd.concat([df3, ddf3], axis=1)

In [38]:
df03.tail()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted July 20.,,730,No ratings yet,True,:D
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 2.', 'las...",,Posted July 2.,,440,No ratings yet,True,so much fun :D


ANALISIS DE SENTIMIENTOS

In [83]:
def analizar_sentimiento(text):
    sentimiento = TextBlob(text)
    return sentimiento.sentiment.polarity
       

CREACION DE COLUMNA DE ANALISIS DE SENTIMIENTO

In [85]:
df03['review'] = df03['review'].astype(str)

df03['Polaridad del sentimiento'] = df03['review'].apply(analizar_sentimiento)

df03['Sentimiento'] = df03['Polaridad del sentimiento'].apply(
    lambda x: 2 if x > 0 else (1 if x == 0 else 0)
)


In [86]:
df03

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Polaridad del sentimiento,Sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.050000,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.114583,2
1,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,-0.036111,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,0.233333,2
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,0.142500,2
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,0.087500,2
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la...",,Posted July 20.,,730,No ratings yet,True,:D,1.000000,2


CREACION DEL 3 ARCHIVO A PARQUET PARA QUE CONSUMA MENOS ESPACIO

In [87]:
df03.to_parquet('r_items.parquet', index=False)

# LECTURA DE LOS ARCHIVOS PARA UNIRLOS 

In [4]:
df2 = pd.read_parquet("u_items.parquet")
df2

,user_id,items_count,steam_id,user_url,items,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike...",10.0,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ...",20.0,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'...",30.0,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla...",40.0,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp...",50.0,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus...",373330.0,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To...",388490.0,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1...",521570.0,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes...",519140.0,Minds Eyes,3.0,3.0


BORRANDO COLUMNAS INNESESARIAS

In [5]:
df2 = df2.drop(columns=['item_id', 'items', 'user_url'])

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 6 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   item_name         object 
 4   playtime_forever  float64
 5   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 236.7+ MB


LECTURA DEL 3 ARCHIVO QUE HABIAMOS CONVERTIDO A PARQUET

In [7]:
df3 = pd.read_parquet("r_items.parquet")
df3

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review,Polaridad del sentimiento,Sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,0.050000,2
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '15 of 20 people (75%...",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,0.114583,2
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'helpful': '0 of 1 people (0%) f...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,-0.036111,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,0.233333,2
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,0.142500,2
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ...",1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,0.087500,2
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'helpful': 'No ratings yet', 'it...",,Posted July 20.,,730,No ratings yet,True,:D,1.000000,2


In [8]:
df3 = df3.drop(columns=['user_url', 'funny', 'posted', 'helpful', 'last_edited', 'reviews'])

In [9]:
df3["item_id"] = df3["item_id"].astype(float)

In [10]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   user_id                    59333 non-null  object 
 1   item_id                    59305 non-null  float64
 2   recommend                  59305 non-null  object 
 3   review                     59333 non-null  object 
 4   Polaridad del sentimiento  59333 non-null  float64
 5   Sentimiento                59333 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 2.7+ MB


# UNION DEL PRIMER Y SEGUNDO DF LEIDOS

In [11]:
df123= pd.merge(df2, df3, on='user_id')

In [12]:
df123.head(2)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2


In [13]:
df1 = pd.read_parquet('I_items.parquet')
df1.head(2)

,publisher,genres,title,release_date,tags,price,item_id,developer
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980.0,Secret Level SRL


MODIFICACION DE NOMBRE DE COLUMNAS PARA PODERLOS UNIR

In [14]:
DataSet_Final = pd.merge(df123, df1, on='item_id')

CREACION DE UNA COLUMNA A FORMATO FECHA INDEPENDIENTE

In [15]:
DataSet_Final['anio'] = pd.to_datetime(DataSet_Final["release_date"], errors='coerce')

In [16]:
DataSet_Final.head(2)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,['Action'],Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2,ACE Team,"['Action', 'Indie']",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21


In [17]:
DataSet_Final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454908 entries, 0 to 6454907
Data columns (total 19 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   user_id                    object        
 1   items_count                int64         
 2   steam_id                   object        
 3   item_name                  object        
 4   playtime_forever           float64       
 5   playtime_2weeks            float64       
 6   item_id                    float64       
 7   recommend                  object        
 8   review                     object        
 9   Polaridad del sentimiento  float64       
 10  Sentimiento                int64         
 11  publisher                  object        
 12  genres                     object        
 13  title                      object        
 14  release_date               datetime64[ns]
 15  tags                       object        
 16  price                      float64  

In [18]:
DataSet_Final['genres'] = DataSet_Final['genres'].str.strip("[]").str.replace("'", "")

In [19]:
DataSet_Final.reset_index(drop=True)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2,ACE Team,"Action, Indie",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
2,76561197970982479,277,76561197970982479,Team Fortress Classic,0.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
3,76561197970982479,277,76561197970982479,Team Fortress Classic,0.0,0.0,22200.0,True,It's unique and worth a playthrough.,0.337500,2,ACE Team,"Action, Indie",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009-04-21
4,76561197970982479,277,76561197970982479,Day of Defeat,7.0,0.0,1250.0,True,Simple yet with great replayability. In my opi...,0.174444,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009-05-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6454903,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,1998-11-08,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...",9.99,Valve,1998-11-08
6454904,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,2015-05-05,"[Early Access, FPS, Remake, Action, Sci-fi, Fi...",19.99,Crowbar Collective,2015-05-05
6454905,LydiaMorley,0,76561198313816521,None,NaN,NaN,273110.0,True,had so much fun plaing this and collecting res...,0.087500,2,Nexon Europe GmbH,"Action, Casual, Free to Play, Massively Multip...",Counter-Strike Nexon: Zombies,2014-10-07,"[Free to Play, Zombies, Multiplayer, FPS, Shoo...",0.00,Nexon,2014-10-07
6454906,LydiaMorley,0,76561198313816521,None,NaN,NaN,730.0,True,:D,1.000000,2,Valve,Action,Counter-Strike: Global Offensive,2012-08-21,"[FPS, Multiplayer, Shooter, Action, Team-Based...",14.99,Valve,2012-08-21


In [21]:
DataSet_Final = DataSet_Final.drop(columns=['tags', 'release_date'])

In [22]:
DataSet_Final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454908 entries, 0 to 6454907
Data columns (total 17 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   user_id                    object        
 1   items_count                int64         
 2   steam_id                   object        
 3   item_name                  object        
 4   playtime_forever           float64       
 5   playtime_2weeks            float64       
 6   item_id                    float64       
 7   recommend                  object        
 8   review                     object        
 9   Polaridad del sentimiento  float64       
 10  Sentimiento                int64         
 11  publisher                  object        
 12  genres                     object        
 13  title                      object        
 14  price                      float64       
 15  developer                  object        
 16  anio                       datetime6

# Creacion de columna year para los endpoints, y medicion de tipos de archivos

In [23]:
DataSet_Final['anio'] = DataSet_Final['anio'].dt.year


In [24]:
DataSet_Final.tail(15)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,item_id,recommend,review,Polaridad del sentimiento,Sentimiento,publisher,genres,title,price,developer,anio
6454893,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454894,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0
6454895,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,9.99,Valve,1998.0
6454896,76561198312638244,36,76561198312638244,ShareX,1.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,19.99,Crowbar Collective,2015.0
6454897,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454898,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0
6454899,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,70.0,True,a must have classic from steam definitely wort...,0.233333,2,Valve,Action,Half-Life,9.99,Valve,1998.0
6454900,76561198312638244,36,76561198312638244,Devil Daggers,0.0,0.0,362890.0,True,this game is a perfect remake of the original ...,0.142500,2,Crowbar Collective,"Action, Adventure, Indie, Early Access",Black Mesa,19.99,Crowbar Collective,2015.0
6454901,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,233270.0,True,this is a very fun and nice 80s themed shooter...,0.249091,2,Ubisoft,"Action, Adventure",Far Cry 3 - Blood Dragon,14.99,Ubisoft Montreal,2013.0
6454902,76561198312638244,36,76561198312638244,DOOM,1178.0,0.0,130.0,True,if you liked Half life i would really recommen...,0.193333,2,Valve,Action,Half-Life: Blue Shift,4.99,Gearbox Software,2001.0


In [25]:


DataSet_Final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454908 entries, 0 to 6454907
Data columns (total 17 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user_id                    object 
 1   items_count                int64  
 2   steam_id                   object 
 3   item_name                  object 
 4   playtime_forever           float64
 5   playtime_2weeks            float64
 6   item_id                    float64
 7   recommend                  object 
 8   review                     object 
 9   Polaridad del sentimiento  float64
 10  Sentimiento                int64  
 11  publisher                  object 
 12  genres                     object 
 13  title                      object 
 14  price                      float64
 15  developer                  object 
 16  anio                       float64
dtypes: float64(6), int64(2), object(9)
memory usage: 837.2+ MB


In [26]:
DataSet_Final.to_parquet("DataSet_Final.parquet")

In [2]:
DataSet_Final = pd.read_parquet('DataSet_Final2.parquet')

In [40]:
formatos["genres"] = formatos["genres"].astype(str)
formatos["developer"] = formatos["developer"].astype(str)
formatos["item_name"] = formatos["item_name"].astype(str)
formatos["developer"] = formatos["developer"].astype(str)

In [43]:
formatos.to_parquet('DataSet_Final2.parquet')

In [26]:
DataSet_Final=DataSet_Final.fillna("")

In [ ]:
df1 = df.drop(columns=['reviews', 'publisher', 'relase_date'], axis=1)

In [39]:
DataSet_Final.head(2)

,user_id,items_count,steam_id,item_name,playtime_forever,playtime_2weeks,reviews,item_id,recommend,Sentimiento,publisher,genres,title,release_date,tags,price,developer,anio
0,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",1250.0,True,2,Tripwire Interactive,Action,Killing Floor,2009-05-14,"[FPS, Zombies, Co-op, Survival, Action, Multip...",19.99,Tripwire Interactive,2009.0
1,76561197970982479,277,76561197970982479,Counter-Strike,6.0,0.0,"{'funny': '', 'helpful': 'No ratings yet', 'it...",22200.0,True,2,ACE Team,"Action, Indie",Zeno Clash,2009-04-21,"[Action, Indie, Surreal, Beat 'em up, FPS, Fig...",9.99,ACE Team,2009.0


In [46]:
def recomendacion(id_producto: int):
    if id_producto not in muestra['steam_id'].values:
        return {'mensaje': 'No existe el id del juego.'}
    
    # Obtener géneros del juego con el id_producto
    generos = muestra.columns[2:17]  # Obtener los nombres de las columnas de género
    
    # Filtrar el dataframe para incluir juegos con géneros coincidentes pero con títulos diferentes
    filtered_df = muestra[(muestra[generos] == 1).any(axis=1) & (muestra['steam_id'] != id_producto)]
    
    # Calcular similitud del coseno
    tdfid_matrix_filtered = tfidf.transform(filtered_df['reviews'])
    cosine_similarity_filtered = linear_kernel(tdfid_matrix_filtered, tdfid_matrix_filtered)
    
    idx = muestra[muestra['steam_id'] == id_producto].index[0]
    sim_cosine = list(enumerate(cosine_similarity_filtered[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = filtered_df['title'].iloc[sim_ind].values.tolist()
    
    return print({'juegos recomendados': list(sim_juegos)})


In [ ]:
def recomendacion_juego(id_juego: int):
    if id_juego not in muestra['id'].values:
        return {'mensaje': 'No existe el id del juego.'}
    titulo = muestra.loc[muestra['id'] == id_juego, 'title'].iloc[0]
    idx = muestra[muestra['title'] == titulo].index[0]
    sim_cosine = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_cosine, key=lambda x: x[1], reverse=True)
    sim_ind = [i for i, _ in sim_scores[1:6]]
    sim_juegos = muestra['title'].iloc[sim_ind].values.tolist()
    return {'juegos recomendados': list(sim_juegos)}